# Getting started

## Introduction

**MIPLearn** is an open source framework that uses machine learning (ML) to accelerate the performance of both commercial and open source mixed-integer programming solvers (e.g. Gurobi, CPLEX, XPRESS, Cbc or SCIP). In this tutorial, we will:

1. Install the Python/Pyomo version of MIPLearn
2. Model a simple optimization problem using JuMP
3. Generate training data and train the ML models
4. Use the ML models together Gurobi to solve new instances

<div class="alert alert-info">
Note
    
The Python/Pyomo version of MIPLearn is currently only compatible with with Gurobi, CPLEX and XPRESS. For broader solver compatibility, see the Julia/JuMP version of the package.
</div>

<div class="alert alert-warning">
Warning
    
MIPLearn is still in early development stage. If run into any bugs or issues, please submit a bug report in our GitHub repository. Comments, suggestions and pull requests are also very welcome!
    
</div>


## Installation

MIPLearn is available in two versions:

- Python version, compatible with the Pyomo modeling language,
- Julia version, compatible with the JuMP modeling language.

In this tutorial, we will demonstrate how to use and install the Python/Pyomo version of the package. The first step is to install Python 3.8+ in your computer. See the [official Python website for more instructions](https://www.python.org/downloads/). After Python is installed, we proceed to install MIPLearn using `pip`:

In [1]:
# !pip install MIPLearn==0.2.0.dev13

In addition to MIPLearn itself, we will also install Gurobi 9.5, a state-of-the-art commercial MILP solver. This step also install a demo license for Gurobi, which should able to solve the small optimization problems in this tutorial. A paid license is required for solving large-scale problems.

In [2]:
!pip install --upgrade -i https://pypi.gurobi.com 'gurobipy>=9.5,<9.6'

Looking in indexes: https://pypi.gurobi.com


<div class="alert alert-info">
    
Note
    
In the code above, we install specific version of all packages to ensure that this tutorial keeps running in the future, even when newer (and possibly incompatible) versions of the packages are released. This is usually a recommended practice for all Python projects.
    
</div>

## Modeling a simple optimization problem

To illustrate how can MIPLearn be used, we will model and solve a small optimization problem related to power systems optimization. The problem we discuss below is a simplification of the **unit commitment problem,** a practical optimization problem solved daily by electric grid operators around the world. 

Suppose that you work at a utility company, and that it is your job to decide which electrical generators should be online at a certain hour of the day, as well as how much power should each generator produce. More specifically, assume that your company owns $n$ generators, denoted by $g_1, \ldots, g_n$. Each generator can either be online or offline. An online generator $g_i$ can produce between $p^\text{min}_i$ to $p^\text{max}_i$ megawatts of power, and it costs your company $c^\text{fix}_i + c^\text{var}_i y_i$, where $y_i$ is the amount of power produced. An offline generator produces nothing and costs nothing. You also know that the total amount of power to be produced needs to be exactly equal to the total demand $d$ (in megawatts). To minimize the costs to your company, which generators should be online, and how much power should they produce?

This simple problem can be modeled as a *mixed-integer linear optimization* problem as follows. For each generator $g_i$, let $x_i \in \{0,1\}$ be a decision variable indicating whether $g_i$ is online, and let $y_i \geq 0$ be a decision variable indicating how much power does $g_i$ produce. The problem is then given by:

$$
\begin{align}
\text{minimize } \quad & \sum_{i=1}^n \left( c^\text{fix}_i x_i + c^\text{var}_i y_i \right) \\
\text{subject to } \quad & y_i \leq p^\text{max}_i x_i & i=1,\ldots,n \\
& y_i \geq p^\text{min}_i x_i & i=1,\ldots,n \\
& \sum_{i=1}^n y_i = d \\
& x_i \in \{0,1\} & i=1,\ldots,n \\
& y_i \geq 0 & i=1,\ldots,n
\end{align}
$$

<div class="alert alert-info">
    
Note
    
We use a simplified version of the unit commitment problem in this tutorial just to make it easier to follow. MIPLearn can also handle realistic, large-scale versions of this problem. See benchmarks for more details.
    
</div>

Next, let us convert this abstract mathematical formulation into a concrete optimization model, using Python and Pyomo. We start by defining a data class `UnitCommitmentData`, which holds all the input data.

In [3]:
from dataclasses import dataclass
import numpy as np

@dataclass
class UnitCommitmentData:
    demand: float
    pmin: np.ndarray
    pmax: np.ndarray
    cfix: np.ndarray
    cvar: np.ndarray

Next, we write a `build_uc_model` function, which converts the input data into a concrete Pyomo model.

In [4]:
import pyomo.environ as pe

def build_uc_model(data: UnitCommitmentData) -> pe.ConcreteModel:
    model = pe.ConcreteModel()
    n = len(data.pmin)
    model.x = pe.Var(range(n), domain=pe.Binary)
    model.y = pe.Var(range(n), domain=pe.NonNegativeReals)
    model.obj = pe.Objective(
        expr=sum(
            data.cfix[i] * model.x[i] +
            data.cvar[i] * model.y[i]
            for i in range(n)
        )
    )
    model.eq_max_power = pe.ConstraintList()
    model.eq_min_power = pe.ConstraintList()
    for i in range(n):
        model.eq_max_power.add(model.y[i] <= data.pmax[i] * model.x[i])
        model.eq_min_power.add(model.y[i] >= data.pmin[i] * model.x[i])
    model.eq_demand = pe.Constraint(
        expr=sum(model.y[i] for i in range(n)) == data.demand,
    )
    return model

At this point, we can already use Pyomo and any mixed-integer linear programming solver to find optimal solutions to any instance of this problem. To illustrate this, let us solve a small instance with three generators:

In [5]:
model = build_uc_model(
    UnitCommitmentData(
        demand = 100.0,
        pmin = [10, 20, 30],
        pmax = [50, 60, 70],
        cfix = [700, 600, 500],
        cvar = [1.5, 2.0, 2.5],
    )
)

solver = pe.SolverFactory("gurobi_persistent")
solver.set_instance(model)
solver.solve()
print("obj =", model.obj())
print("x =", [model.x[i].value for i in range(3)])
print("y =", [model.y[i].value for i in range(3)])

Set parameter Threads to value 1
Set parameter Seed to value 42
Restricted license - for non-production use only - expires 2023-10-25
obj = 1320.0
x = [-0.0, 1.0, 1.0]
y = [0.0, 60.0, 40.0]


Running the code above, we found that the optimal solution for our small problem instance costs \$1320. It is achieve by keeping generators 2 and 3 online and producing, respectively, 60 MW and 40 MW of power.

## Generating training data

Although Gurobi could solve the small example above in a fraction of a second, it gets slower for larger and more complex versions of the problem. If this is a problem that needs to be solved frequently, as it is often the case in practice, it could make sense to spend some time upfront generating a **trained** version of Gurobi, which can solve new instances (similar to the ones it was trained on) faster.

In the following, we will use MIPLearn to train machine learning models that is able to predict the optimal solution for instances that follow a given probability distribution, then it will provide this predicted solution to Gurobi as a warm start. Before we can train the model, we need to collect training data by solving a large number of instances. In real-world situations, we may construct these training instances based on historical data. In this tutorial, we will construct them using a random instance generator:

In [6]:
from scipy.stats import uniform
from typing import List
import random

def random_uc_data(samples: int, n: int, seed: int = 42) -> List[UnitCommitmentData]:
    random.seed(seed)
    np.random.seed(seed)
    pmin = uniform(loc=100_000.0, scale=400_000.0).rvs(n)
    pmax = pmin * uniform(loc=2.0, scale=2.5).rvs(n)
    cfix = pmin * uniform(loc=100.0, scale=25.0).rvs(n)
    cvar = uniform(loc=1.25, scale=0.25).rvs(n)
    return [
        UnitCommitmentData(
            demand = pmax.sum() * uniform(loc=0.5, scale=0.25).rvs(),
            pmin = pmin,
            pmax = pmax,
            cfix = cfix,
            cvar = cvar,
        )
        for i in range(samples)
    ]

In this example, for simplicity, only the demands change from one instance to the next. We could also have randomized the costs, production limits or even the number of units. The more randomization we have in the training data, however, the more challenging it is for the machine learning models to learn solution patterns.

Now we generate 500 instances of this problem, each one with 50 generators, and we use 450 of these instances for training. After generating the instances, we write them to individual files. MIPLearn uses files during the training process because, for large-scale optimization problems, it is often impractical to hold in memory the entire training data, as well as the concrete Pyomo models. Files also make it much easier to solve multiple instances simultaneously, potentially even on multiple machines. We will cover parallel and distributed computing in a future tutorial. The code below generates the files `uc/train/00000.pkl.gz`, `uc/train/00001.pkl.gz`, etc., which contain the input data in compressed (gzipped) pickle format.

In [7]:
from miplearn import save
data = random_uc_data(samples=500, n=50)
train_files = save(data[0:450], "uc/train/")
test_files  = save(data[450:500], "uc/test/")

Finally, we use `LearningSolver` to solve all the training instances. `LearningSolver` is the main component provided by MIPLearn, which integrates MIP solvers and ML. The optimal solutions, along with other useful training data, are stored in HDF5 files `uc/train/00000.h5`, `uc/train/00001.h5`, etc.

In [12]:
from miplearn import LearningSolver
solver = LearningSolver()
solver.solve(train_files, build_uc_model);

## Solving test instances

With training data in hand, we can now fit the ML models, using the `LearningSolver.fit` method, then solve the test instances with `LearningSolver.solve`, as shown below. The `tee=True` parameter asks MIPLearn to print the solver log to the screen.

In [9]:
solver_ml = LearningSolver()
solver_ml.fit(train_files, build_uc_model)
solver_ml.solve(test_files[0:1], build_uc_model, tee=True);

Set parameter LogFile to value "/tmp/tmpvbaqbyty.log"
Set parameter QCPDual to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 16 physical cores, 32 logical processors, using up to 1 threads
Optimize a model with 101 rows, 100 columns and 250 nonzeros
Model fingerprint: 0x8de73876
Coefficient statistics:
  Matrix range     [1e+00, 2e+06]
  Objective range  [1e+00, 6e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+07, 2e+07]
Presolve removed 100 rows and 50 columns
Presolve time: 0.00s
Presolved: 1 rows, 50 columns, 50 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.7349081e+08   1.044003e+04   0.000000e+00      0s
       1    6.8268465e+08   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  6.826846503e+08
Set parameter LogFile to value ""
Set parameter LogFile to value "/tmp/tmp48j6n35b.log"
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 

By examining the solve log above, specifically the line `Loaded user MIP start with objective...`, we can see that MIPLearn was able to construct an initial solution which turned out to be the optimal solution to the problem. Now let us repeat the code above, but using an untrained solver. Note that the `fit` line is omitted.

In [10]:
solver_baseline = LearningSolver()
solver_baseline.solve(test_files[0:1], build_uc_model, tee=True);

Set parameter LogFile to value "/tmp/tmp3uhhdurw.log"
Set parameter QCPDual to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 16 physical cores, 32 logical processors, using up to 1 threads
Optimize a model with 101 rows, 100 columns and 250 nonzeros
Model fingerprint: 0x8de73876
Coefficient statistics:
  Matrix range     [1e+00, 2e+06]
  Objective range  [1e+00, 6e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+07, 2e+07]
Presolve removed 100 rows and 50 columns
Presolve time: 0.00s
Presolved: 1 rows, 50 columns, 50 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.7349081e+08   1.044003e+04   0.000000e+00      0s
       1    6.8268465e+08   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.826846503e+08
Set parameter LogFile to value ""
Set parameter LogFile to value "/tmp/tmp18aqg2ic.log"
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 

In the log above, the `MIP start` line is missing, and Gurobi had to start with a significantly inferior initial solution. The solver was still able to find the optimal solution at the end, but it required using its own internal heuristic procedures. In this example, because we solve very small optimization problems, there was almost no difference in terms of running time. For larger problems, however, the difference can be significant. See benchmarks for more details.

<div class="alert alert-info">
Note
    
In addition to partial initial solutions, MIPLearn is also able to predict lazy constraints, cutting planes and branching priorities. See the next tutorials for more details.
</div>

<div class="alert alert-info">
Note
    
It is not necessary to specify what ML models to use. MIPLearn, by default, will try a number of classical ML models and will choose the one that performs the best, based on k-fold cross validation. MIPLearn is also able to automatically collect features based on the MIP formulation of the problem and the solution to the LP relaxation, among other things, so it does not require handcrafted features. If you do want to customize the models and features, however, that is also possible, as we will see in a later tutorial.
</div>

## Accessing the solution

In the example above, we used `LearningSolver.solve` together with data files to solve both the training and the test instances. The optimal solutions were saved to HDF5 files in the train/test folders, and could be retrieved by reading theses files, but that is not very convenient. In the following example, we show how to build and solve a Pyomo model entirely in-memory, using our trained solver.

In [11]:
# Construct model using previously defined functions
data = random_uc_data(samples=1, n=50)[0]
model = build_uc_model(data)

# Solve model using ML + Gurobi
solver_ml.solve(model)

# Print part of the optimal solution
print("obj =", model.obj())
print(" x =", [model.x[i].value for i in range(5)])
print(" y =", [model.y[i].value for i in range(5)])

obj = 903865807.3536932
 x = [1.0, 1.0, 1.0, 1.0, 1.0]
 y = [1105176.593734543, 1891284.5155055337, 1708177.4224033852, 1438329.610189608, 535496.3347187206]
